# Qwen3-0.6B: дообучение на анекдотах

Скрипты в каталоге `scripts/`. Этот ноутбук фиксирует команды, использованные для scraping → SFT датасета → LoRA-трейна → генерации.


In [ ]:
# 1) Сбор анекдотов
!python scripts/scrape_jokes.py --pages 200 --pause 0.2 0.5 --out data/jokes_raw.txt

# 2) Подготовка SFT датасета
!python scripts/build_sft_dataset.py --input data/jokes_raw.txt --out data/sft_dataset.jsonl --min_len 32 --max_len 400


In [ ]:
# 3) Обучение LoRA (FP16)
!python scripts/train_lora.py --model_id hf_models/Qwen3-0.6B --trust_remote_code --epochs 2 --batch_size 2 --grad_accum 8 --lr 1e-4 --max_length 512 --output_dir outputs/qwen3-jokes-lora
# лог тренировки см. stdout: loss ~1.53 на 2 эпохах, train_runtime ~873s


In [ ]:
# 4) Генерация анекдотов (300+ строк, max_new_tokens=64/96)
!python scripts/generate_jokes.py --base_model hf_models/Qwen3-0.6B --adapter outputs/qwen3-jokes-lora --out outputs/jokes_final.txt --num 300 --trust_remote_code


## Логи трейна (фрагмент)

```
trainable params: 10,092,544 || all params: 606,142,464 || trainable%: 1.6650
{'loss': 3.0304, ... 'epoch': 0.17}
{'loss': 1.7649, ... 'epoch': 0.34}
{'loss': 1.5274, ... 'epoch': 0.51}
{'loss': 1.3305, ... 'epoch': 0.84}
{'loss': 1.317,  ... 'epoch': 1.17}
{'loss': 1.264,  ... 'epoch': 1.51}
{'loss': 1.258,  ... 'epoch': 2.00}
{'train_runtime': 873.0, 'train_samples_per_second': 2.17, 'train_steps_per_second': 0.137, 'train_loss': 1.53, 'epoch': 2.0}
```

## Примечания
- Базовая модель лежит локально: `hf_models/Qwen3-0.6B/model.safetensors`
- Итоговые анекдоты: `outputs/jokes_final.txt` (549 строк)
